In [5]:
import random
import torch
from torch.autograd import Function
from torch.autograd import Variable
from scipy.signal import convolve2d, correlate2d
from torch.nn.modules.module import Module 
from torch.nn.parameter import Parameter


In [2]:
class DynamicNet(torch.nn.Module):
  def __init__(self, D_in, H, D_out):
    """
    In the constructor we construct three nn.Linear instances that we will use
    in the forward pass.
    """
    super(DynamicNet, self).__init__()
    self.input_linear = torch.nn.Linear(D_in, H)
    self.middle_linear = torch.nn.Linear(H, H)
    self.output_linear = torch.nn.Linear(H, D_out)

  def forward(self, x):
    """
    For the forward pass of the model, we randomly choose either 0, 1, 2, or 3
    and reuse the middle_linear Module that many times to compute hidden layer
    representations.

    Since each forward pass builds a dynamic computation graph, we can use normal
    Python control-flow operators like loops or conditional statements when
    defining the forward pass of the model.

    Here we also see that it is perfectly safe to reuse the same Module many
    times when defining a computational graph. This is a big improvement from Lua
    Torch, where each Module could be used only once.
    """
    h_relu = self.input_linear(x).clamp(min=0)
    for _ in range(random.randint(0, 3)):
      h_relu = self.middle_linear(h_relu).clamp(min=0)
    y_pred = self.output_linear(h_relu)
    return y_pred


# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs, and wrap them in Variables
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# Construct our model by instantiating the class defined above
model = DynamicNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(size_average=False)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
for t in range(500):
  # Forward pass: Compute predicted y by passing x to the model
  y_pred = model(x)

  # Compute and print loss
  loss = criterion(y_pred, y)
  print(t, loss.data[0])

  # Zero gradients, perform a backward pass, and update the weights.
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

(0, 732.5946044921875)
(1, 676.2997436523438)
(2, 683.893798828125)
(3, 681.8519897460938)
(4, 469.0211181640625)
(5, 408.431396484375)
(6, 678.7081298828125)
(7, 677.68798828125)
(8, 673.5112915039062)
(9, 675.185302734375)
(10, 668.5148315429688)
(11, 206.9691619873047)
(12, 671.1268920898438)
(13, 165.8954315185547)
(14, 668.1176147460938)
(15, 666.1344604492188)
(16, 663.5509643554688)
(17, 92.0317153930664)
(18, 592.6405639648438)
(19, 640.1012573242188)
(20, 649.8265991210938)
(21, 532.0264892578125)
(22, 73.52848052978516)
(23, 595.9910888671875)
(24, 445.9525146484375)
(25, 608.4380493164062)
(26, 591.6952514648438)
(27, 506.4504699707031)
(28, 540.6077880859375)
(29, 292.7088317871094)
(30, 131.9813690185547)
(31, 377.5372314453125)
(32, 234.17576599121094)
(33, 381.8858337402344)
(34, 350.0209045410156)
(35, 194.21656799316406)
(36, 265.5238342285156)
(37, 243.4134063720703)
(38, 155.14788818359375)
(39, 208.06373596191406)
(40, 147.27505493164062)
(41, 216.2056884765625)
(42

In [6]:
class ScipyConv2dFunction(Function):

    def forward(self, input, filter):
        result = correlate2d(input.numpy(), filter.numpy(), mode='valid')
        self.save_for_backward(input, filter)
        return torch.FloatTensor(result)

    def backward(self, grad_output):
        input, filter = self.saved_tensors
        grad_input = convolve2d(grad_output.numpy(), filter.t().numpy(), mode='full')
        grad_filter = convolve2d(input.numpy(), grad_output.numpy(), mode='valid')
        return torch.FloatTensor(grad_input), torch.FloatTensor(grad_filter)


class ScipyConv2d(Module):

    def __init__(self, kh, kw):
        super(ScipyConv2d, self).__init__()
        self.filter = Parameter(torch.randn(kh, kw))

    def forward(self, input):
        return ScipyConv2dFunction()(input, self.filter)

In [ ]:
module = ScipyConv2d(3, 3)
print(list(module.parameters()))
input = Variable(torch.randn(10, 10), requires_grad=True)
output = module(input)
print(output)
output.backward(torch.randn(8, 8))
print(input.grad)